# Lab Exercise: SQL Analysis with Polars

> Add blockquote



In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [48]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values="NA")
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values="NA")
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values="NA", schema_overrides={"precip": pl.Float64, "visib": pl.Float64})

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-905584215.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [49]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines;
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [50]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10;

""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [51]:
# Write your SQL query here
result = ctx.execute("""SELECT *
FROM flights
WHERE dep_delay > 120;
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [52]:
# Write your SQL query here
result = ctx.execute("""SELECT origin, AVG(dep_delay) AS avg_dep_delay
FROM flights
GROUP BY origin;

""")
print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ JFK    ┆ 12.112159     │
│ EWR    ┆ 15.107954     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [53]:
# First, let's check what columns are available
result = ctx.execute("""SELECT month, COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1;
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [54]:
# Write your SQL query here
result = ctx.execute("""SELECT carrier,
       SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS on_time_rate
FROM flights
GROUP BY carrier;


""")
print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ YV      ┆ 64.725458    │
│ AA      ┆ 82.437594    │
│ UA      ┆ 78.043126    │
│ AS      ┆ 86.554622    │
│ VX      ┆ 82.099961    │
│ …       ┆ …            │
│ OO      ┆ 71.875       │
│ EV      ┆ 65.922138    │
│ MQ      ┆ 75.171421    │
│ B6      ┆ 76.537018    │
│ FL      ┆ 71.687117    │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [55]:
# Write your SQL query here
result = ctx.execute("""SELECT f.*, a.name AS airline_name
FROM flights f
JOIN airlines a
  ON f.carrier = a.carrier;
""")
print(result)

shape: (336_776, 20)
┌──────┬───────┬─────┬──────────┬───┬──────┬────────┬───────────────────────┬──────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ hour ┆ minute ┆ time_hour             ┆ airline_name         │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---  ┆ ---    ┆ ---                   ┆ ---                  │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64  ┆ i64    ┆ datetime[μs, UTC]     ┆ str                  │
╞══════╪═══════╪═════╪══════════╪═══╪══════╪════════╪═══════════════════════╪══════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00   ┆ United Air Lines     │
│      ┆       ┆     ┆          ┆   ┆      ┆        ┆ UTC                   ┆ Inc.                 │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00   ┆ United Air Lines     │
│      ┆       ┆     ┆          ┆   ┆      ┆        ┆ UTC                   ┆ Inc.                 │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 5    ┆ 40     ┆ 2013-01-01 10:00

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [56]:
# Write your SQL query here
result = ctx.execute("""SELECT f.carrier,
       AVG(f.year - p.year) AS avg_plane_age
FROM flights f
JOIN planes p
  ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier;

""")

print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ OO      ┆ 6.84375       │
│ 9E      ┆ 7.101053      │
│ DL      ┆ 16.372169     │
│ UA      ┆ 13.207691     │
│ MQ      ┆ 35.319        │
│ …       ┆ …             │
│ YV      ┆ 9.313758      │
│ EV      ┆ 11.308998     │
│ AS      ┆ 3.33662       │
│ US      ┆ 9.103663      │
│ VX      ┆ 4.473643      │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [57]:
# First, explore the weather table structure
result = ctx.execute("""SELECT f.*
FROM flights f
JOIN weather w
  ON f.year = w.year
 AND f.month = w.month
 AND f.day = w.day
 AND f.hour = w.hour
 AND f.origin = w.origin
WHERE f.dep_delay > 30
  AND (w.precip > 0.1 OR w.visib < 1 OR w.wind_speed > 20);
""")
print(result)

shape: (5_603, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 2217     ┆ … ┆ 1005     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 2326     ┆ … ┆ 2248     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1238     ┆ … ┆ 820      ┆ 11   ┆ 59     ┆ 2013-01-04 16:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1400     ┆ … ┆ 212      ┆ 13   ┆ 20     ┆ 2013-01-04 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 4   ┆ 1455     ┆ … ┆ 187      ┆ 14   ┆ 22     ┆ 2013-01-04 19:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [58]:
# Write your SQL query here
result = ctx.execute("""SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS num_flights
FROM flights f
JOIN planes p
    ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY num_flights DESC
LIMIT 10;


""")
print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────────┐
│ manufacturer                  ┆ model           ┆ num_flights │
│ ---                           ┆ ---             ┆ ---         │
│ str                           ┆ str             ┆ u32         │
╞═══════════════════════════════╪═════════════════╪═════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278       │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553       │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051       │
│ BOEING                        ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807       │
│ BOEING                        ┆ 737-7H4         ┆ 10389       │
│ BOEING                        ┆ 757-222         ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932     

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [60]:
# Write your SQL query here
result = ctx.execute("""
WITH FlightRputeDetails AS (
    SELECT
        f.dep_delay,
        o.name AS origin_name,
        d.name AS destination_name
    FROM flights AS f
    JOIN airports AS o ON f.origin = o.faa
    JOIN airports AS d ON f.dest = d.faa
    Where f.dep_delay IS NOT NULL
    )
    SELECT
        origin_name,
        destination_name,
        COUNT(*) AS total_flights,
        AVG(dep_delay) AS avg_departure_delay,
        CAST(SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) AS REAL) * 100/COUNT(*) AS percentage_delayed_more_than_30
        FROM FlightRputeDetails
    GROUP BY origin_name, destination_name
    ORDER BY total_flights DESC
    LIMIT 10;
""")
print(result)

shape: (10, 5)
┌────────────────┬─────────────────────┬───────────────┬─────────────────────┬─────────────────────┐
│ origin_name    ┆ destination_name    ┆ total_flights ┆ avg_departure_delay ┆ percentage_delayed_ │
│ ---            ┆ ---                 ┆ ---           ┆ ---                 ┆ more_than_3…        │
│ str            ┆ str                 ┆ u32           ┆ f64                 ┆ ---                 │
│                ┆                     ┆               ┆                     ┆ f64                 │
╞════════════════╪═════════════════════╪═══════════════╪═════════════════════╪═════════════════════╡
│ John F Kennedy ┆ Los Angeles Intl    ┆ 11196         ┆ 8.522508            ┆ 9.88746             │
│ Intl           ┆                     ┆               ┆                     ┆                     │
│ La Guardia     ┆ Hartsfield Jackson  ┆ 10082         ┆ 11.448621           ┆ 12.467764           │
│                ┆ Atlanta Int…        ┆               ┆                    

In [ ]:
# Write your SQL query here
result = ctx.execute("""
-- Your query here
""")

# print(result)

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [ ]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here